In [1]:
#! pip install pydeequ

In [2]:
#! apt-get install wget -y

In [3]:
#!wget https://repo1.maven.org/maven2/com/amazon/deequ/deequ/1.2.2-spark-3.0/deequ-1.2.2-spark-3.0.jar

In [4]:
! pwd

/opt/workspace/test


In [5]:
! ls -l *.jar

-rw-r--r-- 1 root root 1258590 Apr 28  2021 deequ-1.2.2-spark-3.0.jar


In [6]:
from pyspark.sql import SparkSession
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite

# Initialize the Spark session
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("DEEQU").\
        master("spark://spark-master:7077").\
config("spark.jars", "/opt/workspace/test/deequ-1.2.2-spark-3.0.jar").\
        config("spark.executor.memory", "3000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        getOrCreate()

Please set env variable SPARK_VERSION
23/04/24 15:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
spark

In [12]:
# Sample data for demonstration purposes
data = spark.createDataFrame([
    (1, "A", 100),
    (2, "B", None),
    (3, "C", 200),
    (4, None, 300),
    (5, "E", None)
], ["id", "name", "value"])

In [14]:
data.show()

+---+----+-----+
| id|name|value|
+---+----+-----+
|  1|   A|  100|
|  2|   B| null|
|  3|   C|  200|
|  4|null|  300|
|  5|   E| null|
+---+----+-----+



In [26]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, ApproxCountDistinct, Completeness, Compliance, Mean, Size

analysisResult = AnalysisRunner(spark).onData(data).addAnalyzer(Size()).addAnalyzer(Completeness("name")).addAnalyzer(ApproxCountDistinct("name"))\
.addAnalyzer(Mean("value")).addAnalyzer(Compliance("value", "value > 200")).run()

In [27]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+-------------------+-----+
| entity|instance|               name|value|
+-------+--------+-------------------+-----+
|Dataset|       *|               Size|  5.0|
| Column|   value|         Compliance|  0.2|
| Column|    name|       Completeness|  0.8|
| Column|    name|ApproxCountDistinct|  4.0|
| Column|   value|               Mean|200.0|
+-------+--------+-------------------+-----+

